In [4]:
import requests
import locale
from lxml import html
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError
from pprint import pprint
from datetime import date

locale.setlocale(locale.LC_TIME, 'ru')

'ru'

In [13]:
client = MongoClient('localhost', 27017)
db = client['news']
my_news = db.my_news

In [14]:
def add_data(doc, only_new = True):
    exists = my_news.find_one({'link': doc['link']})
    if only_new == True:
        if not exists:
            try:
                my_news.insert_one(doc)
            except DuplicateKeyError:
                pass
    else:
        try:
            my_news.insert_one(doc)
        except DuplicateKeyError:
            pass

In [15]:
headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"}
url = 'http://lenta.ru'
session = requests.Session()

response = session.get(url+'/parts/news', headers = headers)

print(f'URL is {response.url}')
print(f'Status code is {response.status_code}')

dom = html.fromstring(response.text)

URL is https://lenta.ru/parts/news
Status code is 200


In [16]:
items = dom.xpath("//li[@class='parts-page__item']")
for item in items:
    news = {}
    resource = url
    name = item.xpath(".//h3[@class='card-full-news__title']/text()")
    link = item.xpath(".//h3[@class='card-full-news__title']/../@href")
    news_time = item.xpath(".//time[contains(@class, 'card-full-news__date')]/text()")
    
    news['resource'] = resource
    news['name'] = ''.join(name)
    
    if 'https' in str(link):
        news['link'] = ''.join(link)
    else: 
        news['link'] = url + ''.join(link)
    if len(''.join(news_time)) < 10:
        news['news_time'] = ''.join(news_time) + ', ' + date.today().strftime("%d %B %Y")
        
    else:
        news['news_time'] = ''.join(news_time)
        
    add_data(news, only_new=True)

In [17]:
for item in my_news.find({'resource': url}):
    pprint(item)

{'_id': ObjectId('64cbc13b73b2c9e09ca821df'),
 'link': 'http://lenta.ru/news/2023/08/03/dachniki_kalin/',
 'name': 'Власти российского города захотели отнять дома у дачников',
 'news_time': '18:00, 03 Август 2023',
 'resource': 'http://lenta.ru'}
{'_id': ObjectId('64cbc13b73b2c9e09ca821e0'),
 'link': 'http://lenta.ru/news/2023/08/03/vorobyevskaya/',
 'name': 'Собянин рассказал о благоустройстве еще одной набережной в Москве',
 'news_time': '17:59, 03 Август 2023',
 'resource': 'http://lenta.ru'}
{'_id': ObjectId('64cbc13b73b2c9e09ca821e1'),
 'link': 'http://lenta.ru/news/2023/08/03/lechenie/',
 'name': 'Российского педофила-рецидивиста осудили на 16 лет и назначили ему '
         'лечение',
 'news_time': '17:58, 03 Август 2023',
 'resource': 'http://lenta.ru'}
{'_id': ObjectId('64cbc13b73b2c9e09ca821e2'),
 'link': 'http://lenta.ru/news/2023/08/03/ready/',
 'name': 'В Госдуме отреагировали на призыв прекратить госзакупки иномарок для '
         'чиновников',
 'news_time': '17:57, 03 Авг